### Get unique users

In [1]:
import pandas as pd
import pickle

/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [34]:
followers = pd.read_pickle('follower_query.pkl')
following = pd.read_pickle('following_queries.pkl')

print(type(followers), type(following))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [35]:
followers.head()

,follower_of,user_they_follow,relationship
0,215624405,477797010,is_following
1,110105365,477797010,is_following
2,460374543,477797010,is_following
3,1048158,477797010,is_following
4,425616225,477797010,is_following


In [9]:
follower_users = list(followers['follower_of']) + list(followers['user_they_follow'])
#follower_users

In [45]:
following.head()

,user,is_followed_by,relationship
0,38611053,477797010,is_followed_by
1,281433942,477797010,is_followed_by
2,18088787,477797010,is_followed_by
3,326424092,477797010,is_followed_by
4,103880112,477797010,is_followed_by


In [12]:
following_users = list(following.user) + list(following.is_followed_by)
users = list(set(follower_users + following_users))

In [13]:
len(users)

3352957

In [14]:
with open('unique_users.pkl', 'wb') as file_name:
    pickle.dump(users, file_name)
    file_name.close()

Great! Now that I have a list of all the unique users, I just need to create a follower matrix ^__^

### This is me creating the follower matrix.

In [73]:
from copy import deepcopy

followers_relash = deepcopy(followers)
following_relash = deepcopy(following)

In [74]:
followers_relash.rename(columns = {
    
    'follower_of': 'user',
    'user_they_follow': 'is_following'
}, inplace = True)

followers_relash.head()

,user,is_following,relationship
0,215624405,477797010,is_following
1,110105365,477797010,is_following
2,460374543,477797010,is_following
3,1048158,477797010,is_following
4,425616225,477797010,is_following


In [75]:
#1 if a if following b
followers_relash.relationship = 1 # if present and if not, then the sparse matrix will make it 0

In [76]:
followers_relash.head()

,user,is_following,relationship
0,215624405,477797010,1
1,110105365,477797010,1
2,460374543,477797010,1
3,1048158,477797010,1
4,425616225,477797010,1


In [77]:
following_relash.head()

,user,is_followed_by,relationship
0,38611053,477797010,is_followed_by
1,281433942,477797010,is_followed_by
2,18088787,477797010,is_followed_by
3,326424092,477797010,is_followed_by
4,103880112,477797010,is_followed_by


In [78]:
following_relash.rename(columns = {
    'user': 'is_following'    
}, inplace = True)
following_relash.rename(columns = {
    
    'is_followed_by':'user'
}, inplace= True) # this helps to make sure that the df merge correctly
#following_relash['relationship'] = 1
following_relash.relationship = 1
# following_relash = following_relash[['user', 'is_following', 'relationship']] #rearranges the columns
following_relash.head()

,is_following,user,relationship
0,38611053,477797010,1
1,281433942,477797010,1
2,18088787,477797010,1
3,326424092,477797010,1
4,103880112,477797010,1


In [79]:
following_relash = following_relash[['user', 'is_following', 'relationship']]
following_relash.head()

,user,is_following,relationship
0,477797010,38611053,1
1,477797010,281433942,1
2,477797010,18088787,1
3,477797010,326424092,1
4,477797010,103880112,1


### Now that I have configured the dataframe to how I want it, I can create the spare matrix

In [80]:
following_relash.head()

,user,is_following,relationship
0,477797010,38611053,1
1,477797010,281433942,1
2,477797010,18088787,1
3,477797010,326424092,1
4,477797010,103880112,1


In [81]:
followers_relash.head()

,user,is_following,relationship
0,215624405,477797010,1
1,110105365,477797010,1
2,460374543,477797010,1
3,1048158,477797010,1
4,425616225,477797010,1


In [82]:
print(len(following_relash) + len(followers_relash))
combined = pd.concat([following_relash, followers_relash], axis = 0)
print(len(combined)) # compare the two print statements to make sure that you concatenated it correctly

9826739
9826739


In [83]:
import pickle

with open('combined_following_relationships.pkl', 'wb') as file_name:
    pickle.dump(combined, file_name)
    file_name.close()

In [84]:
combined.head()

,user,is_following,relationship
0,477797010,38611053,1
1,477797010,281433942,1
2,477797010,18088787,1
3,477797010,326424092,1
4,477797010,103880112,1


Please read the relationship column/dataframe as 'user A is following user B'

In [ ]:
# to make a pivot table

movie_matrixmovie_ma  = combined.pivot(index='user', columns='is_following', values='relationship').fillna(0)
movie_matrix.head()